In [1]:
from pathlib import Path
import pandas as pd
import sys
import gc

# Example command: python merge_and_process_compustat.py C:\Users\jasonjia\Dropbox\Projects\conference_call\output\04_match_firm_names_to_gvkeys\04.1_process_compustat_and_hassan_files\compustat_processed\20220705 C:\Users\jasonjia\Dropbox\Projects\conference_call\output\04_match_firm_names_to_gvkeys\04.1_process_compustat_and_hassan_files\compustat_processed\20220705\ciqcompany_mergedwithgvkeyandcountry.csv

In [2]:
inputfolder = Path(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\output\04_match_firm_names_to_gvkeys\04.1_process_compustat_and_hassan_files\compustat_processed\20220705")
outputfilepath = Path(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\output\04_match_firm_names_to_gvkeys\04.1_process_compustat_and_hassan_files\compustat_processed\20220705\ciqcompany_mergedwithgvkeyandcountry.csv")
 
inputfilename_ciqcompany = 'ciqcompany.csv'
inputfilename_wrds_gvkey = 'wrds_gvkey.csv'
inputfilename_ciqcountrygeo = 'ciqcountrygeo.csv'

In [3]:
# Import files
ciqcompany = pd.read_csv(Path(inputfolder / inputfilename_ciqcompany))
wrds_gvkey = pd.read_csv(Path(inputfolder / inputfilename_wrds_gvkey))
ciqcountrygeo = pd.read_csv(Path(inputfolder / inputfilename_ciqcountrygeo))

In [4]:
def get_number_of_duplicated_values_in_a_df_col(df_col):
    value_counts = df_col.value_counts()
    return value_counts[value_counts > 1].count()

In [5]:
# Columns
# ciqcompany.columns: ['companyid', 'companyname', 'countryid']
# wrds_gvkey.columns: ['companyid', 'gvkey', 'startdate', 'enddate', 'companyname']
# ciqcountrygeo.columns: ['countryid', 'country', 'isocountry2', 'isocountry3', 'regionid', 'region']

# Filter wrds_gvkey and ciqcountrygeo to keep relevant columns
wrds_gvkey = wrds_gvkey[['companyid', 'gvkey']]
ciqcountrygeo = ciqcountrygeo[['countryid','country']]

In [6]:
# Check for unique companyids and gvkeys -> There will be some companyids without gvkeys.
print("Number of unique companyids:", ciqcompany['companyid'].nunique())
print("Number of unique gvkeys:", wrds_gvkey['gvkey'].nunique())

Number of unique companyids: 27593141
Number of unique gvkeys: 115553


In [7]:
# Check for duplicates.
print("Checks for wrds_gvkey and ciqcountrygeo:")
print("Number of duplicated companyids in wrds_gvkey:", get_number_of_duplicated_values_in_a_df_col(wrds_gvkey['companyid']))
print("Number of duplicated gvkeys in wrds_gvkey:", get_number_of_duplicated_values_in_a_df_col(wrds_gvkey['gvkey']))
print("Number of duplicated countryids in ciqcountrygeo:", get_number_of_duplicated_values_in_a_df_col(ciqcountrygeo['countryid']))
print("Number of duplicated countries in ciqcountrygeo:", get_number_of_duplicated_values_in_a_df_col(ciqcountrygeo['country']))

# Some company names have multiple companyids - so 1 gvkey can be linked to multiple companyids. 
# But 1 companyid should be linked to only 1 gvkey. So we remove duplicates for companyid.
# The ciqcountrygeo dataset should be perfectly 1:1, i.e. no duplicated countryid or country.

Checks for wrds_gvkey and ciqcountrygeo:
Number of duplicated companyids in wrds_gvkey: 494
Number of duplicated gvkeys in wrds_gvkey: 4896
Number of duplicated countryids in ciqcountrygeo: 0
Number of duplicated countries in ciqcountrygeo: 0


In [8]:
# Some companyids have multiple gvkeys. We keep only the first gvkey.
print("Dropped duplicates for companyid in wrds_gvkey, redoing checks:")
wrds_gvkey = wrds_gvkey.drop_duplicates('companyid')
print("Number of duplicated companyids in wrds_gvkey:", get_number_of_duplicated_values_in_a_df_col(wrds_gvkey['companyid']))
print("Number of duplicated gvkeys in wrds_gvkey:", get_number_of_duplicated_values_in_a_df_col(wrds_gvkey['gvkey']))
print("Number of duplicated countryids in ciqcountrygeo:", get_number_of_duplicated_values_in_a_df_col(ciqcountrygeo['countryid']))
print("Number of duplicated countries in ciqcountrygeo:", get_number_of_duplicated_values_in_a_df_col(ciqcountrygeo['country']))


Dropped duplicates for companyid in wrds_gvkey, redoing checks:
Number of duplicated companyids in wrds_gvkey: 0
Number of duplicated gvkeys in wrds_gvkey: 4812
Number of duplicated countryids in ciqcountrygeo: 0
Number of duplicated countries in ciqcountrygeo: 0


In [9]:
# Idea: Merge ciqcompany with gvkey and country
# 1. Merge ciqcompany with gvkey - 1 companyid in ciqcompany <=> 1 company id in wrds_gvkey
ciqcompany = ciqcompany.merge(wrds_gvkey, on='companyid', how='left', validate='1:1')
gc.collect() # Delete collectable items to clear memory

# 2. Merge (ciqcompany, gvkey) with country - 1 countryid in ciqcompany => 1 countryid in ciqcountrygeo
ciqcompany = ciqcompany.merge(ciqcountrygeo, on='countryid', how='left', validate='m:1')

In [10]:
# Check: Number of rows without gvkeys
print("Number of rows of ciqcompany (merged) without gvkeys:", ciqcompany['gvkey'].isna().sum())

# Drop rows with NaN values for gvkeys
ciqcompany = ciqcompany.dropna(subset=['gvkey'])
print("Dropped NaN values for gvkey, checking stats:")
print("Number of rows of ciqcompany (merged):", ciqcompany.shape[0])
print("Number of unique gvkeys in ciqcompany (merged):", ciqcompany['gvkey'].nunique())
print("Number of unique companyids in ciqcompany (merged):", ciqcompany['companyid'].nunique())
print("Number of unique companynames in ciqcompany (merged):", ciqcompany['companyname'].nunique())
print("Number of duplicated companynames in ciqcompany (merged):", get_number_of_duplicated_values_in_a_df_col(ciqcompany['companyname']))

# Some companynames are duplicated because the company has branches in multiple countries. So each country's branch has its own companyid and countryid. 
# Some companynames just have multiple companyids even with the same countryid.
# In both cases, we only want the companyname, not the id. So we take the first companyname.

Number of rows of ciqcompany (merged) without gvkeys: 27472708
Dropped NaN values for gvkey, checking stats:
Number of rows of ciqcompany (merged): 120433
Number of unique gvkeys in ciqcompany (merged): 115150
Number of unique companyids in ciqcompany (merged): 120433
Number of unique companynames in ciqcompany (merged): 119662
Number of duplicated companynames in ciqcompany (merged): 754


In [11]:
# Drop duplicates for company name, and recheck statistics
ciqcompany = ciqcompany.drop_duplicates('companyname')
print("Dropped duplicates for companyname, rechecking stats:")
print("Number of rows of ciqcompany (merged):", ciqcompany.shape[0])
print("Number of unique gvkeys in ciqcompany (merged):", ciqcompany['gvkey'].nunique())
print("Number of unique companyids in ciqcompany (merged):", ciqcompany['companyid'].nunique())
print("Number of unique companynames in ciqcompany (merged):", ciqcompany['companyname'].nunique())
print("Number of duplicated companynames in ciqcompany (merged):", get_number_of_duplicated_values_in_a_df_col(ciqcompany['companyname']))

Dropped duplicates for companyname, rechecking stats:
Number of rows of ciqcompany (merged): 119662
Number of unique gvkeys in ciqcompany (merged): 114783
Number of unique companyids in ciqcompany (merged): 119662
Number of unique companynames in ciqcompany (merged): 119662
Number of duplicated companynames in ciqcompany (merged): 0


In [12]:
ciqcompany

companyid                                        companyname  \
5         1.850700e+04                                      2M Invest A/S   
7         1.851100e+04                                       3i Group plc   
14        1.852700e+04                                            ABB Ltd   
70        1.867100e+04                              Albemarle Corporation   
86        1.871100e+04                           The Allstate Corporation   
...                ...                                                ...   
27569993  1.784731e+09                                        Borouge plc   
27570484  1.784741e+09                   ADC Acquisition Corporation PJSC   
27575304  1.785110e+09  SBI Mutual Fund - SBI Debt Fund Series C - 46 ...   
27575305  1.785110e+09  SBI Mutual Fund - SBI Debt Fund Series C - 35 ...   
27581939  1.785478e+09  Tianhong Asset Management Co. Ltd. - CSI Star ...   

          countryid     gvkey               country  
5              55.0  235716.0               Denmark  
7             212.0  210835.0        United Kingdom  
14            195.0  210418.0           Switzerland  
70            213.0   29751.0         United States  
86            213.0   28349.0         United States  
...             ...       ...                   ...  
27569993      211.0  353390.0  United Arab Emirates  
27570484      211.0  353372.0  United Arab Emirates  
27575304       93.0  353418.0                 India  
27575305       93.0  353419.0                 India  
27581939       43.0  353453.0                 China  

[119662 rows x 5 columns]

In [13]:
# Save the merged df into csv
ciqcompany.to_csv(outputfilepath, index = False)
print("Saved merged and processed compustat files to:", outputfilepath)

Saved merged and processed compustat files to: C:\Users\jasonjia\Dropbox\Projects\conference_call\output\04_match_firm_names_to_gvkeys\04.1_process_compustat_and_hassan_files\compustat_processed\20220705\ciqcompany_mergedwithgvkeyandcountry.csv
